# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from api_keys import weather_api_key
from api_keys import geoapify_key
gmaps.configure(api_key= geoapify_key)

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/WeatherPy.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mildura,-34.2000,142.1500,59.07,57,79,19.17,AU,1667354064
1,1,pacific grove,36.6177,-121.9166,55.08,84,0,8.99,US,1667353922
2,2,bredasdorp,-34.5322,20.0403,60.85,88,95,15.28,ZA,1667353634
3,3,tiarei,-17.5333,-149.3333,73.42,91,100,21.72,PF,1667353631
4,4,hobart,-42.8794,147.3294,52.27,58,75,12.66,AU,1667353627


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
locations = city_data_df[['Lat', 'Lng']]
weights = city_data_df['Humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
plt.savefig("Humidity_map.png")

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data=city_data_df

# A max temperature lower than 75 degrees but higher than 65.
filtered_city_data = filtered_city_data.loc[(filtered_city_data["Max Temp"] < 75) & (filtered_city_data["Max Temp"] > 65)]

# Wind speed less than 10 mph.
filtered_city_data = filtered_city_data.loc[filtered_city_data["Wind Speed"] < 10]

# Zero cloudiness.
filtered_city_data = filtered_city_data.loc[filtered_city_data["Cloudiness"] == 0]

# Drop any rows with null values
filtered_city_data = filtered_city_data.dropna()

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,araouane,18.9048,-3.5265,74.12,17,0,8.81,ML,1667354119
143,143,cape town,-33.9258,18.4232,65.26,53,0,5.75,ZA,1667353512
157,157,luzhou,28.8724,105.4405,70.18,56,0,3.06,CN,1667354128
238,238,saint-pierre,-21.3393,55.4781,70.92,68,0,5.75,RE,1667354159
320,320,guerrero negro,27.9769,-114.0611,69.24,71,0,9.98,MX,1667354189
359,359,myitkyina,25.3833,97.4000,71.67,59,0,1.63,MM,1667354204
380,380,khamaria,23.2167,79.8833,67.42,77,0,3.44,IN,1667354213
401,401,wuan,31.6836,112.0044,65.79,54,0,2.53,CN,1667354219
428,428,mayo,38.8876,-76.5119,65.17,82,0,4.61,US,1667354226
473,473,alice springs,-23.7000,133.8833,67.55,26,0,9.22,AU,1667353658


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
133,araouane,ML,18.9048,-3.5265,17,
143,cape town,ZA,-33.9258,18.4232,53,
157,luzhou,CN,28.8724,105.4405,56,
238,saint-pierre,RE,-21.3393,55.4781,68,
320,guerrero negro,MX,27.9769,-114.0611,71,
359,myitkyina,MM,25.3833,97.4000,59,
380,khamaria,IN,23.2167,79.8833,77,
401,wuan,CN,31.6836,112.0044,54,
428,mayo,US,38.8876,-76.5119,82,
473,alice springs,AU,-23.7000,133.8833,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
params = {
    "radius": 10000,
    "type": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
   # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
araouane - nearest hotel: No hotel found
cape town - nearest hotel: Radisson Blu Hotel Waterfront, Cape Town
luzhou - nearest hotel: Shijichun Hotel
saint-pierre - nearest hotel: Alize Plage
guerrero negro - nearest hotel: The Halfway Inn
myitkyina - nearest hotel: Shwe Phyu Hotel
khamaria - nearest hotel: Hotel Vasu
wuan - nearest hotel: 武镇大酒店
mayo - nearest hotel: Resorts Resource Group International
alice springs - nearest hotel: Desert Palms Alice Springs
tomatlan - nearest hotel: El Salvial
okmulgee - nearest hotel: Holiday Inn Express & Suites Okmulgee, an IHG Hotel


In [40]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
133,araouane,ML,18.9048,-3.5265,17,No hotel found
143,cape town,ZA,-33.9258,18.4232,53,"Radisson Blu Hotel Waterfront, Cape Town"
157,luzhou,CN,28.8724,105.4405,56,Shijichun Hotel
238,saint-pierre,RE,-21.3393,55.4781,68,Alize Plage
320,guerrero negro,MX,27.9769,-114.0611,71,The Halfway Inn
359,myitkyina,MM,25.3833,97.4000,59,Shwe Phyu Hotel
380,khamaria,IN,23.2167,79.8833,77,Hotel Vasu
401,wuan,CN,31.6836,112.0044,54,武镇大酒店
428,mayo,US,38.8876,-76.5119,82,Resorts Resource Group International
473,alice springs,AU,-23.7000,133.8833,26,Desert Palms Alice Springs


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """

Hotel Name
{Hotel Name}

City
{City}
Country
{Country}

"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
plt.savefig("map_plot.png")

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>